In [56]:
import os, warnings
import pandas as pd
from konlpy.tag import Okt
from collections import Counter

warnings.filterwarnings('ignore')

In [57]:
url = 'UseData/Voucher/Cleaned/'
csvUrl = 'UseData/Voucher/WordExtract/'
os.listdir(url), os.listdir(csvUrl)

(['네이버쇼핑_(단호박_다이어트식품)_cleaned.csv',
  '네이버쇼핑_(단호박_샐러드)_cleaned.csv',
  '네이버쇼핑_(단호박칩_반려동물)_cleaned.csv',
  '스마트스토어_(구운_단호박)_cleaned.csv',
  '스마트스토어_(다이어트_스낵)_cleaned.csv',
  '스마트스토어_(호박_스낵)_cleaned.csv'],
 ['네이버쇼핑_(단호박_다이어트식품)_cleaned_keyword_neg.csv',
  '네이버쇼핑_(단호박_다이어트식품)_cleaned_keyword_pos.csv',
  '네이버쇼핑_(단호박_다이어트식품)_cleaned_keyword_total.csv',
  '네이버쇼핑_(단호박_샐러드)_cleaned_keyword_neg.csv',
  '네이버쇼핑_(단호박_샐러드)_cleaned_keyword_pos.csv',
  '네이버쇼핑_(단호박_샐러드)_cleaned_keyword_total.csv',
  '네이버쇼핑_(단호박칩_반려동물)_cleaned_keyword_neg.csv',
  '네이버쇼핑_(단호박칩_반려동물)_cleaned_keyword_pos.csv',
  '네이버쇼핑_(단호박칩_반려동물)_cleaned_keyword_total.csv',
  '스마트스토어_(구운_단호박)_cleaned_keyword_neg.csv',
  '스마트스토어_(구운_단호박)_cleaned_keyword_pos.csv',
  '스마트스토어_(구운_단호박)_cleaned_keyword_total.csv',
  '스마트스토어_(다이어트_스낵)_cleaned_keyword_neg.csv',
  '스마트스토어_(다이어트_스낵)_cleaned_keyword_pos.csv',
  '스마트스토어_(다이어트_스낵)_cleaned_keyword_total.csv',
  '스마트스토어_(호박_스낵)_cleaned_keyword_neg.csv',
  '스마트스토어_(호박_스낵)_cleaned_keyword_pos.csv',
  '스마

In [58]:
# 어짜피 내용이 똑같아서 굳이 나눌 필요가 없을거같은데
filesDF = pd.DataFrame({'File' : os.listdir(url)})
filesDF

,File
0,네이버쇼핑_(단호박_다이어트식품)_cleaned.csv
1,네이버쇼핑_(단호박_샐러드)_cleaned.csv
2,네이버쇼핑_(단호박칩_반려동물)_cleaned.csv
3,스마트스토어_(구운_단호박)_cleaned.csv
4,스마트스토어_(다이어트_스낵)_cleaned.csv
5,스마트스토어_(호박_스낵)_cleaned.csv


    긍/부정 별 키워드 csv 파일 생성 함수
        -> JJS Tokenizing 04 참고.

In [77]:
# 사전 로딩
stop_neg = 'UseData/Voucher/stoplist_neg.txt'
stop_pos = 'UseData/Voucher/stoplist_pos.txt'

# 긍정사전 로딩
with open (stop_pos, 'r', encoding='utf8') as f:
    stopwords_pos = []
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        stopwords_pos.append(line)
        
# 부정사전 로딩
with open(stop_neg, 'r', encoding='utf8') as f2:
    stopwords_neg = []
    lines2 = f2.readlines()
    for line in lines2:
        line = line.strip()
        stopwords_neg.append(line)

# 이 사전을 같이 합친다? set 연산.
temp = list((set(stopwords_pos)).difference(set(stopwords_neg))) # 차집합을 리스트로 구하고
stopwords = stopwords_neg + temp # 빈 부분(교집합 부분)을 더하기.
len(stopwords), len(stopwords_pos), len(stopwords_neg) # 약 중복되는 40개가 없어진듯.

(219, 115, 144)

In [82]:
stopwords_pos[-3], stopwords_neg[-3], stopwords[-1]

('단호박', '단호박', '바중')

In [83]:
# 앞서 정의한 불용어 사전 이용, 불용어 제거.
def StopWordsEliminator(dataframe):
    for words in dataframe['word']:
        if words in stopwords:
            dataframe.drop(dataframe[dataframe['word'] == words].index, inplace=True) # 불용어 row 제거
            dataframe = dataframe.reset_index(drop=True) # 인덱스 재배치.
    return dataframe

In [84]:
# 단어 추출
def WordSelector(series):
    wordList = []
    for index in range(len(series)):
        for word, pos in series[index]:
            if (pos == 'Noun') or (pos == 'Adjective'):
                wordList.append(word)
    return wordList

In [85]:
okt = Okt()

def CSVMaker(url, file):
    df = pd.read_csv(url + file, index_col=0) # 파일 1개씩 읽어와서
    df.drop('제목', axis=1, inplace=True) # '제목' 컬럼은 사용 X.
    
    # 형태소 분리 및 태깅작업 먼저 진행
    df['Contents'] = df['Contents'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    
    # 긍/부정 프레임 나누기
    df_pos = df.loc[df['긍부정']==1].reset_index()
    df_pos.drop('index', axis=1, inplace=True)
    df_neg = df.loc[df['긍부정']==0].reset_index()
    df_neg.drop('index', axis=1, inplace=True)
    
    # 긍/부정 별로 데이터 프레임 생성
    # 긍정은 500개 정도? 부정은 전부. (부정이 갯수가 더 적으니까.)
    wordPosList = Counter(WordSelector(df_pos['Contents'])).most_common(500)
    wordPosDF = pd.DataFrame(wordPosList)
    wordPosDF.columns = wordPosDF.columns.astype(str)
    wordPosDF = wordPosDF.rename(columns={'0':'word', '1':'freq'})
    # 길이가 1인 단어들은 전부 제거
    wordPosDF['TF'] = wordPosDF['word'].apply(lambda x: len(x) > 1)
    wordPosDF.drop(wordPosDF[wordPosDF['TF'] == False].index, inplace=True)
    wordPosDF = wordPosDF.reset_index(drop=True)
    wordPosDF.drop('TF', axis=1, inplace=True)
    wordPosDF = StopWordsEliminator(wordPosDF) # 사전을 통한 불용어 제거
    
    wordNegList = Counter(WordSelector(df_neg['Contents'])).most_common()
    wordNegDF = pd.DataFrame(wordNegList)
    wordNegDF.columns = wordNegDF.columns.astype(str)
    wordNegDF = wordNegDF.rename(columns={'0':'word', '1':'freq'})
    # 길이가 1인 단어들은 전부 제거
    wordNegDF['TF'] = wordNegDF['word'].apply(lambda x: len(x) > 1)
    wordNegDF.drop(wordNegDF[wordNegDF['TF'] == False].index, inplace=True)
    wordNegDF = wordNegDF.reset_index(drop=True)
    wordNegDF.drop('TF', axis=1, inplace=True)
    wordNegDF = StopWordsEliminator(wordNegDF)
    
    # CSV파일로 저장
    wordPosDF.to_csv(csvUrl + file[:-4] + '_keyword_pos.csv', index=True, header=True, encoding='utf-8-sig')
    wordNegDF.to_csv(csvUrl + file[:-4] + '_keyword_neg.csv', index=True, header=True, encoding='utf-8-sig')

In [49]:
file = filesDF['File'][0]
CSVMaker(url, file) # 만들어지는건 확인.

In [86]:
for index in range(len(filesDF)):
    file = filesDF['File'][index]
    CSVMaker(url, file)

    긍/부정 나눈거 말고도 그냥 통합본을 하나 만들까
    키워드 한 500개 정도만 뽑아서?

In [87]:
def TotalCSVMaker(url, file):
    df = pd.read_csv(url + file, index_col=0) # 파일 1개씩 읽어와서
    df.drop('제목', axis=1, inplace=True) # '제목' 컬럼은 사용 X.
    
    # 형태소 분리 및 태깅작업 먼저 진행
    df['Contents'] = df['Contents'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    wordList = Counter(WordSelector(df['Contents'])).most_common(500)
    wordDF = pd.DataFrame(wordList)
    wordDF.columns = wordDF.columns.astype(str)
    wordDF = wordDF.rename(columns={'0':'word', '1':'freq'})
    wordDF['TF'] = wordDF['word'].apply(lambda x: len(x) > 1)
    wordDF.drop(wordDF[wordDF['TF'] == False].index, inplace=True)
    wordDF = wordDF.reset_index(drop=True)
    wordDF.drop('TF', axis=1, inplace=True)
    wordDF = StopWordsEliminator(wordDF)
    
    wordDF.to_csv(csvUrl + file[:-4] + '_keyword_total.csv', index=True, header=True, encoding='utf-8-sig')

In [ ]:
for index in range(len(filesDF)):
    file = filesDF['File'][index]
    TotalCSVMaker(url, file)

    단어-빈도 CSV 만들었고 이제
    
    1. WordCloud 그려보고
    2. Word2Vec 돌려서 -> '단호박'처럼 특정 키워드 넣고 연관된 단어가 뭐 나오는지 확인?
    3. LDA 돌려보기
        ----- 2,3번은 모두 토큰화 진행된 단어들로 해야함.